# Chapter 3 Solving Non-invertible Linear System

Let us solve $A x = b$ when $A$ is non-square or not of full rank. We will use $LU$-decomposition.

In [1]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [3]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [4]:
def solve_Ax_b(A, b):
    m = A.shape[0]
    n = A.shape[1]
    Aaug = np.hstack((A,-b))
    # LU-decomposition
    P, L, U = sp.linalg.lu(Aaug)
    print(U)

    # find pivot entries
    epsilon = 10**(-10)     # Setting epsilon for negligibles
    pivot = n+1
    rank_u = 0
    pivots = []
    while pivot >= 0:
        non_zero_indices = np.nonzero(U[rank_u,:]**2 > epsilon)
        if np.size(non_zero_indices) == 0 or pivot == non_zero_indices[0][0]:
            pivot = -1
        else:
            pivot = non_zero_indices[0][0]
            pivots.append(pivot)
            rank_u += 1

    print('Rank = ', rank_u)
    print('Pivot variables = ', np.int64(pivots))

    if n in pivots:
        print("The linear system is infeasible since right hand b increases the rank.")
        return
    
    # find column indices of free variables
    free_variables = np.sort(list(set(range(Aaug.shape[1]))- set(pivots)))
    print("Free variables = ", free_variables)

    # Collect pivot columns and free variables
    U_pivot = U[:rank_u, pivots]
    print('pivot columns of U = ')
    print(U_pivot)
    U_free = U[:rank_u, free_variables]
    print('free variable columns of U = ')
    print(U_free)

    # Solve for the pivot part of a basis of the null space of A
    X_pivot = np.linalg.solve(U_pivot,-U_free)
    print('pivot part of X = ')
    print(X_pivot)

    # remove the right hand b from free variables
    free_variables = free_variables[:-1]

    # Combine pivot part and free variable part
    X = np.hstack((np.eye(n)[:,free_variables], np.zeros(n).reshape(-1,1)))
    X[pivots,:] = X_pivot
    print('X = ')
    print(X)

    print(np.allclose(A @ X, np.hstack((np.zeros((m,n-rank_u)),b))))    

In [5]:
# Setting solvability
solvable = 1
# Setting parameters
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)
if solvable:
    b = A @ np.random.randn(n,1)
else:
    b = np.random.randn(m,1)
    
solve_Ax_b(A,b)

[[ 4.7681  0.1044  3.2178 -2.8033  1.1823  1.4959 -2.0964  1.1382 -2.607  -3.101   0.5823  6.7452]
 [ 0.     -1.9489 -1.1444  2.8387 -1.8745  0.8415  2.6641 -1.4921  0.5115  0.3728 -1.3578  3.4253]
 [ 0.      0.     -6.4523 -1.5385 -2.7568 -4.6019  1.095  -5.2944  0.4542  0.3544 -5.5106 11.0692]
 [ 0.      0.      0.     -4.8266 -0.8278 -1.508   3.0865 -3.3227  0.8416  3.2029  1.9995 -4.5756]
 [ 0.      0.      0.      0.      3.2898  0.8499 -6.381  -1.4725 -0.9782 -2.5167 -3.3422  1.5297]
 [ 0.      0.      0.      0.      0.     -0.5253  0.0971  0.4391 -1.5328 -1.355  -0.5281  3.2379]
 [ 0.      0.      0.      0.      0.      0.      0.813   1.3656 -1.1678 -1.8809 -4.6256  4.8846]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.      0.      0.      0.     -0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.     -0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.     -0.     -0.      0.    ]]
Rank =  7